# Empenhos da Cultura Cearense
- Trabalho de Extração de dados
- Disciplina: Introdução à Ciência de Dados
- Estudante: Renato Freitas

## Gastos Públicos no Setor Cultural


Onde | R\$ | 2011 | R\$ | 2018
---|--|---------|--|----------
Gastos nas três esferas | R\$ 7.1 bilhões || R\$ 9.1 bilhões|
Federal | R\$ 1.360.528 | 19.2\% | R\$ 1.925.133 | 21.1\%
Estadual | R\$ 2.270.552 | 32.0\% | R\$ 2.504.970 | 27.5\%
Municipal | R\$ 3.462.388 | 48.8\% | R\$ 4.690.086 | 51.4\%
Despesas dos municípios cearenses | - |3.8\%| - |4.3\%


Fonte: IBGE, Sistema de Informações e Indicadores Culturais. https://biblioteca.ibge.gov.br/visualizacao/livros/liv101687.pdf

## Objetivo
Gerar um dataset de empenhos da cultura cearense dos cinco últimos anos.

## Fonte de Dados
Portal de Sitemas e Consultas Municípios - TCE. https://municipios.tce.ce.gov.br/

<img src="tcm.png" />

##  Como obter os dados.
Módulo Dados Abertos - via API.

Para obter os empenhos é preciso conhecer o código do município, do órgão e informar ano pretendido.
 - PASSO 1. OBTER OS CÓDIGOS DOS MUNICÍPIOS
 - PASSO 2. OBTER OS CÓDIGOS DOS ORGÃO (SECRETARIA DE CULTURA, POR EXEMPLO)
 - PASSO 3. OBTER OS EMPENHOS

### PASSO 1. OBTER OS CÓDIGOS DOS MUNICÍPIOS

In [2]:
# Bibliotecas
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import pandas as pd
import json
import logging
import time

In [3]:
# Armazena todos os municípios do Ceará. 
municipios = []

In [4]:
# API do Tribunal de Contas do Estado do Ceará
api_tce = "https://api.tce.ce.gov.br/index.php/sim/1_0/"

In [5]:
# Rota dos municipios no TCE
rota_municipios = "{}municipios.json?".format(api_tce)

response = requests.get(rota_municipios)
municipios = response.json()["rsp"]["_content"]

print("Municípios: {}".format(len(municipios)))

Municípios: 184


### PASSO 2. OBTER OS CÓDIGOS DOS ORGÃOS 
(SECRETARIAS DE CULTURA, POR EXEMPLO)

In [7]:
#https://api.tce.ce.gov.br/index.php/sim/1_0/unidades_orcamentarias?codigo_municipio=002&exercicio_orcamento=201800

ano_atual = date.today().year
ini_orgaos = time.time()

# Obter todos os orgão nos últimos 4 anos, ou seja, da última gestão.
for municipio in municipios:
    orgao = {}
    orgaos_cultura = []
    for ano in range(ano_atual - 4, ano_atual):
        cod_mun_tce = municipio["codigo_municipio"]
        rota_orgaos = "{}unidades_orcamentarias.json?codigo_municipio={}&exercicio_orcamento={}00".format(api_tce, cod_mun_tce, ano)
    
        response = requests.get(rota_orgaos)
        conteudo = response.text.replace("\\", "\\\\")
        orgaos = json.loads(conteudo)["rsp"]["_content"]
        
        for key in orgaos:
            NOME_ORG = key["nome_unidade"].upper()
            if NOME_ORG.find("CULTURA") != -1 and NOME_ORG.find("AGRICULTURA") == -1 and NOME_ORG.find("AQUICULTURA") == -1:
                orgaos_cultura.append([key["codigo_orgao"], key["nome_unidade"], str(ano)])
                
        municipio["orgaos"] = orgaos_cultura
fim_orgaos = time.time()  
print("Tempo: {}seg.".format(fim_orgaos - ini_orgaos))
municipios

Tempo: 171.3581964969635seg.


[{'codigo_municipio': '002',
  'nome_municipio': 'ABAIARA',
  'geoibgeId': '2300101',
  'orgaos': [['06', 'SECRETARIA DE CULTURA', '2017'],
   ['06', 'Secretaria de Cultura', '2018'],
   ['06', 'Secretaria de Cultura', '2019'],
   ['06', 'Secretaria de Cultura', '2020']]},
 {'codigo_municipio': '003',
  'nome_municipio': 'ACARAPE',
  'geoibgeId': '2300150',
  'orgaos': [['08', 'Secretaria de Cultura, Esporte e Lazer', '2017'],
   ['08', 'Secretaria de Cultura, Esporte e Lazer', '2018'],
   ['08', 'Secretaria de Cultura, Esporte e Lazer', '2019'],
   ['08', 'Secretaria de Cultura, Esporte e Lazer', '2020']]},
 {'codigo_municipio': '004',
  'nome_municipio': 'ACARAU',
  'geoibgeId': '2300200',
  'orgaos': [['09', 'SECRETARIA DE CULTURA E TURISMO', '2017'],
   ['09', 'SECRETARIA DE CULTURA E TURISMO', '2018'],
   ['08', 'Secretaria de Turismo e Cultura', '2019'],
   ['08', 'Secretaria de Turismo e Cultura', '2020']]},
 {'codigo_municipio': '005',
  'nome_municipio': 'ACOPIARA',
  'geoibge

### PASSO 3. OBTER OS EMPENHOS

In [8]:
#https://api.tce.ce.gov.br/index.php/sim/1_0/notas_empenhos?codigo_municipio=014&codigo_orgao=06&data_referencia_empenho=201101
def obtemRotaEmpenhos(cod_mun_ce, cod_org_tce, ano): 
    """Recebe o código do municipio, o código do orgão e o ano"""
    
    return """{}notas_empenhos.json?codigo_municipio={}&codigo_orgao={}&data_referencia_empenho={}01""".format(api_tce, cod_mun_ce, cod_org_tce, ano)

In [9]:
empenhos = []
ini_empenhos = time.time()
for municipio in municipios:
    print(municipio["nome_municipio"])
    for orgao in municipio["orgaos"]:
        url = obtemRotaEmpenhos(municipio["codigo_municipio"], orgao[0], orgao[2])
        response = requests.get(url)
        conteudo = response.text.replace("\\", "\\\\")
        empenhos.extend(json.loads(conteudo, strict=False)["rsp"]["_content"])
        
fim_empenhos = time.time()     
print("Tempo: {}".format(fim_empenhos - ini_empenhos))
len(empenhos)

ABAIARA
ACARAPE
ACARAU
ACOPIARA
AIUABA
ALCANTARAS
ALTANEIRA
ALTO SANTO
AMONTADA
ANTONINA DO NORTE
APUIARES
AQUIRAZ
ARACATI
ARACOIABA
ARARIPE
ARATUBA
ARNEIROZ
ASSARE
AURORA
BAIXIO
BANABUIU
BARBALHA
BARREIRA
BARRO
BARROQUINHA
BATURITE
BEBERIBE
BELA CRUZ
BOA VIAGEM
BREJO SANTO
CAMOCIM
CAMPOS SALES
CANINDE
CAPISTRANO
CARIDADE
CARIRE
CARIRIACU
CARIUS
CARNAUBAL
CASCAVEL
CATARINA
CAUCAIA
CEDRO
CHAVAL
CHOROZINHO
COREAU
CRATEUS
CRATO
CROATA
CRUZ
DEPUTADO IRAPUAN PINHEIRO
ERERE
EUSEBIO
FARIAS BRITO
FORQUILHA
FORTALEZA
FRECHEIRINHA
GENERAL SAMPAIO
GRACA
GRANJA
GRANJEIRO
GROAIRAS
GUAIUBA
GUARACIABA DO NORTE
GUARAMIRANGA
HIDROLANDIA
HORIZONTE
IBARETAMA
IBIAPINA
IBICUITINGA
ICAPUI
ICO
IGUATU
INDEPENDENCIA
IPAPORANGA
IPAUMIRIM
IPU
IPUEIRAS
IRACEMA
IRAUCUBA
ITAICABA
ITAPAJE
ITAPIPOCA
ITAPIUNA
ITAREMA
ITATIRA
JAGUARETAMA
JAGUARIBARA
JAGUARIBE
JAGUARUANA
JARDIM
JATI
JUAZEIRO DO NORTE
JUCAS
LAVRAS DA MANGABEIRA
LIMOEIRO DO NORTE
MADALENA
MARACANAU
MARANGUAPE
MARCO
MARTINOPOLE
MASSAPE
MAURITI
MERUOCA
MILA

20745

In [10]:
empenhos[0]

{'codigo_municipio': '002',
 'exercicio_orcamento': '201700',
 'codigo_orgao': '06',
 'codigo_unidade': '01  ',
 'data_emissao_empenho': '2017-01-02 00:00:00',
 'numero_empenho': '02010005',
 'data_referencia_empenho': '201701',
 'codigo_funcao': '13',
 'codigo_subfuncao': '122',
 'codigo_programa': '0030',
 'codigo_projeto_atividade': '2',
 'numero_projeto_atividade': '023',
 'numero_subprojeto_atividade': '0000',
 'codigo_elemento_despesa': '31901100',
 'modalidade_empenho': 'E',
 'descricao_empenho': 'Valor que se empenha p/ fazer face as despesas com DISPENDIO COM O PESSOAL LOTADOS JUNTO A ESTA SECRETARIA DE CULTURA NO PRESENTE EXERCICIO FINANCEIRO DE 2017, NESTE MUNICIPIO DE ABAIARA.',
 'valor_anterior_saldo_dotacao': '59550.8000',
 'valor_empenhado': '59550.0000',
 'valor_atual_saldo_dotacao': '0.8000',
 'tipo_processo_licitatorio': 'F',
 'numero_documento_negociante': '66666666666',
 'estado_empenho': 'AP',
 'numero_nota_anulacao': ' ',
 'data_emissao_empenho_substituto': '',
 '

## Agregar valor aos empenhos 
Obter situação, atividade primária e código da atividade primária da empresa

Site: https://serenata.ai/

Site|Github
:--:|:--:
<img src="serenata.png" width="600"/>|<img src="git.png" />

Site: https://receitaws.com.br/

ReceitaWS|Planos
:--:|:--:
<img src="receita.png" width=600>|<img src="receita-planos.png">

In [11]:
def cnpjNaoExiste(cnpj, data):
    return {'cnpj': cnpj, 'ultima_atualizacao': data, 'status': 'ERROR', 'message': 'CNPJ rejeitado pela Receita Federal', 'billing': {'free': True, 'database': True}}

In [12]:
# Obter situacao dos CNPJS
# Fonte: https://github.com/okfn-brasil/serenata-de-amor/blob/main/research/src/fetch_cnpj_info.py
logger = logging.getLogger('fetch_cnpj')

def fetch_cnpj_info(cnpj, timeout=60):
    url = 'http://receitaws.com.br/v1/cnpj/%s' % cnpj
    
    try:
        response = requests.get(url,timeout=timeout)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            logger.debug('Sleeping 60 seconds to try again.')
            time.sleep(60)
            response2 = requests.get(url,timeout=timeout)
            print("sleep 60 secs: ",response2.status_code)
            return response2.json()
        else:
            return cnpjNaoExiste(cnpj, date.today())
            print(response.text)
    except requests.exceptions.Timeout as e:
        logger.debug(e)
    except requests.exceptions.ConnectionError as e:
        logger.debug(e)

In [13]:
ex3_situacao = fetch_cnpj_info("00000000053201")

print("ex3: ", ex3_situacao)

ex3:  {'atividade_principal': [{'text': 'Bancos múltiplos, com carteira comercial', 'code': '64.22-1-00'}], 'data_situacao': '03/11/2005', 'tipo': 'FILIAL', 'nome': 'BANCO DO BRASIL SA', 'efr': 'UNIÃO', 'uf': 'CE', 'telefone': '(88) 3643-9900', 'email': 'age0532@bb.com.br', 'atividades_secundarias': [{'text': 'Outras atividades de serviços financeiros não especificadas anteriormente', 'code': '64.99-9-99'}], 'situacao': 'ATIVA', 'bairro': 'CENTRO', 'logradouro': 'AV DOS CONSTITUINTES', 'numero': '68', 'cep': '62.350-000', 'municipio': 'UBAJARA', 'porte': 'DEMAIS', 'abertura': '01/08/1966', 'natureza_juridica': '203-8 - Sociedade de Economia Mista', 'fantasia': 'UBAJARA (CE)', 'cnpj': '00.000.000/0532-01', 'ultima_atualizacao': '2021-03-02T02:00:45.768Z', 'status': 'OK', 'complemento': '', 'motivo_situacao': '', 'situacao_especial': '', 'data_situacao_especial': '', 'capital_social': '0.00', 'qsa': [], 'extra': {}, 'billing': {'free': True, 'database': True}}


In [14]:
cont_cpf = 0
cont_cnpj = 0
for e in empenhos:
    cpfCnpj = str(e["numero_documento_negociante"]).strip()
    if len(cpfCnpj) < 14:
        cont_cpf += 1
    else:
        cont_cnpj += 1

print("{} CPFs + {} CNPJs = {} DOCs".format(cont_cpf, cont_cnpj, cont_cpf+cont_cnpj))    

8030 CPFs + 12715 CNPJs = 20745 DOCs


In [54]:
# Adicionar situacao da Empresa ao dataset
favorecidos = {}
inicio = time.time()
count = 19000
#8161 
for empenho in empenhos[19000:]:
    print(count, end=' ')
    count += 1
    cpfCnpj = str(empenho["numero_documento_negociante"]).strip()
    
    empenho["situacao_favorecido"] = ""
    empenho["data_situacao"] = ""
    empenho["atividade_principal_texto"] = ""
    empenho["atividade_principal_codigo"] = ""
    
    if len(cpfCnpj) == 14:
        
        cnpjJaConsultado = favorecidos.get(cpfCnpj)
        
        if cnpjJaConsultado != None:
            print("CNPJ {} JÁ consultado!".format(cpfCnpj))
            empenho["situacao_favorecido"] = cnpjJaConsultado["situacao"]
            empenho["data_situacao"] = cnpjJaConsultado["data_situacao"]
            empenho["atividade_principal_texto"] = cnpjJaConsultado["atividade_principal"][0]["text"]
            empenho["atividade_principal_codigo"] = cnpjJaConsultado["atividade_principal"][0]["code"]
        else:
#             print("CNPJ {} ainda NÃO consultado".format(cpfCnpj))
            favorecido = fetch_cnpj_info(cpfCnpj)
            if favorecido != None and favorecido["status"].upper() == "OK":
                empenho["situacao_favorecido"] = favorecido["situacao"]
                empenho["data_situacao"] = favorecido["data_situacao"]
                empenho["atividade_principal_texto"] = favorecido["atividade_principal"][0]["text"]
                empenho["atividade_principal_codigo"] = favorecido["atividade_principal"][0]["code"]
                favorecidos[cpfCnpj] = favorecido
            else:
                empenho["situacao_favorecido"] = favorecido["message"]
                empenho["data_situacao"] = favorecido["ultima_atualizacao"]
                empenho["atividade_principal_texto"] = ""
                empenho["atividade_principal_codigo"] = ""
fim = time.time()
print("\nCNPJ's distintos: {}".format(len(favorecidos)))
print("Tempo para Consultar os CNPJs: {}".format(fim - inicio))

19000 19001 19002 19003 19004 19005 19006 19007 19008 19009 19010 19011 19012 19013 19014 19015 19016 19017 sleep 60 secs:  200
19018 19019 19020 19021 19022 19023 19024 sleep 60 secs:  200
19025 19026 19027 19028 19029 CNPJ 02625806000109 JÁ consultado!
19030 19031 19032 19033 19034 19035 19036 19037 19038 sleep 60 secs:  200
19039 19040 19041 19042 sleep 60 secs:  200
19043 19044 19045 19046 19047 19048 19049 CNPJ 26681201000195 JÁ consultado!
19050 CNPJ 26681201000195 JÁ consultado!
19051 CNPJ 26681201000195 JÁ consultado!
19052 sleep 60 secs:  200
19053 19054 19055 CNPJ 26681201000195 JÁ consultado!
19056 19057 19058 19059 19060 19061 19062 19063 19064 19065 19066 19067 19068 19069 19070 19071 19072 19073 19074 19075 19076 19077 19078 19079 19080 sleep 60 secs:  200
19081 19082 19083 19084 19085 sleep 60 secs:  200
19086 19087 CNPJ 00721689000143 JÁ consultado!
19088 CNPJ 02625806000109 JÁ consultado!
19089 19090 19091 sleep 60 secs:  200
19092 19093 19094 19095 19096 19097 19098 1

19429 19430 CNPJ 00360305000104 JÁ consultado!
19431 CNPJ 29979036000140 JÁ consultado!
19432 19433 sleep 60 secs:  200
19434 CNPJ 11993324000194 JÁ consultado!
19435 19436 19437 19438 CNPJ 07040108000157 JÁ consultado!
19439 CNPJ 07040108000157 JÁ consultado!
19440 CNPJ 07047251000170 JÁ consultado!
19441 CNPJ 33000118000179 JÁ consultado!
19442 CNPJ 12021435000100 JÁ consultado!
19443 sleep 60 secs:  200
19444 19445 19446 19447 CNPJ 29979036000140 JÁ consultado!
19448 CNPJ 07047251000170 JÁ consultado!
19449 19450 19451 CNPJ 07047251000170 JÁ consultado!
19452 sleep 60 secs:  200
19453 CNPJ 00360305000104 JÁ consultado!
19454 CNPJ 29979036000140 JÁ consultado!
19455 19456 19457 19458 19459 19460 CNPJ 00000000121142 JÁ consultado!
19461 CNPJ 06043412000195 JÁ consultado!
19462 CNPJ 07040108000157 JÁ consultado!
19463 sleep 60 secs:  200
19464 19465 CNPJ 29979036004219 JÁ consultado!
19466 CNPJ 07040108000157 JÁ consultado!
19467 CNPJ 07047251000170 JÁ consultado!
19468 19469 CNPJ 0704

19847 19848 CNPJ 18866411000120 JÁ consultado!
19849 CNPJ 07673114000141 JÁ consultado!
19850 19851 CNPJ 29979036000140 JÁ consultado!
19852 CNPJ 00000000183920 JÁ consultado!
19853 CNPJ 07040108000157 JÁ consultado!
19854 CNPJ 07047251000170 JÁ consultado!
19855 CNPJ 11528420000161 JÁ consultado!
19856 CNPJ 07779242000174 JÁ consultado!
19857 CNPJ 07673114000141 JÁ consultado!
19858 19859 CNPJ 29979036000140 JÁ consultado!
19860 CNPJ 07047251000170 JÁ consultado!
19861 CNPJ 07040108000157 JÁ consultado!
19862 CNPJ 00000000183920 JÁ consultado!
19863 CNPJ 01300487000190 JÁ consultado!
19864 CNPJ 03773788000167 JÁ consultado!
19865 19866 19867 19868 19869 CNPJ 10653634000105 JÁ consultado!
19870 CNPJ 11528420000161 JÁ consultado!
19871 CNPJ 05578657000154 JÁ consultado!
19872 CNPJ 15754475000140 JÁ consultado!
19873 CNPJ 18866411000120 JÁ consultado!
19874 CNPJ 07673114000141 JÁ consultado!
19875 19876 CNPJ 29979036000140 JÁ consultado!
19877 CNPJ 00000000183920 JÁ consultado!
19878 CNP

20155 20156 20157 20158 20159 20160 20161 20162 CNPJ 29979036000140 JÁ consultado!
20163 sleep 60 secs:  200
20164 CNPJ 07040108000157 JÁ consultado!
20165 CNPJ 07047251000170 JÁ consultado!
20166 CNPJ 63386627000142 JÁ consultado!
20167 20168 20169 20170 20171 20172 20173 20174 20175 20176 20177 20178 20179 20180 20181 20182 20183 20184 20185 20186 20187 20188 20189 20190 20191 20192 CNPJ 00000000217921 JÁ consultado!
20193 CNPJ 60746948184507 JÁ consultado!
20194 CNPJ 00360305075201 JÁ consultado!
20195 20196 20197 20198 20199 20200 CNPJ 29979036000140 JÁ consultado!
20201 CNPJ 13384138000183 JÁ consultado!
20202 CNPJ 07040108000157 JÁ consultado!
20203 CNPJ 07047251000170 JÁ consultado!
20204 CNPJ 63386627000142 JÁ consultado!
20205 CNPJ 19423991000143 JÁ consultado!
20206 20207 20208 20209 20210 20211 20212 20213 20214 20215 20216 20217 20218 20219 20220 20221 20222 20223 20224 20225 20226 20227 20228 20229 20230 CNPJ 00000000217921 JÁ consultado!
20231 20232 20233 20234 20235 CNPJ

20604 20605 20606 20607 20608 CNPJ 07040108000157 JÁ consultado!
20609 CNPJ 29979036004219 JÁ consultado!
20610 CNPJ 29979036004219 JÁ consultado!
20611 CNPJ 07158749000100 JÁ consultado!
20612 CNPJ 09002710000125 JÁ consultado!
20613 CNPJ 07047251000170 JÁ consultado!
20614 CNPJ 07047251000170 JÁ consultado!
20615 CNPJ 31529891000100 JÁ consultado!
20616 CNPJ 26033638000112 JÁ consultado!
20617 20618 CNPJ 15243080000183 JÁ consultado!
20619 CNPJ 07040108000157 JÁ consultado!
20620 CNPJ 07040108000157 JÁ consultado!
20621 CNPJ 00000000462306 JÁ consultado!
20622 CNPJ 13163430000176 JÁ consultado!
20623 CNPJ 13163430000176 JÁ consultado!
20624 20625 CNPJ 31529891000100 JÁ consultado!
20626 20627 20628 CNPJ 07040108000157 JÁ consultado!
20629 CNPJ 29979036004219 JÁ consultado!
20630 CNPJ 29979036004219 JÁ consultado!
20631 CNPJ 07158749000100 JÁ consultado!
20632 CNPJ 09002710000125 JÁ consultado!
20633 CNPJ 07047251000170 JÁ consultado!
20634 CNPJ 07047251000170 JÁ consultado!
20635 CNP

In [55]:
empenhos

[{'codigo_municipio': '002',
  'exercicio_orcamento': '201700',
  'codigo_orgao': '06',
  'codigo_unidade': '01  ',
  'data_emissao_empenho': '2017-01-02 00:00:00',
  'numero_empenho': '02010005',
  'data_referencia_empenho': '201701',
  'codigo_funcao': '13',
  'codigo_subfuncao': '122',
  'codigo_programa': '0030',
  'codigo_projeto_atividade': '2',
  'numero_projeto_atividade': '023',
  'numero_subprojeto_atividade': '0000',
  'codigo_elemento_despesa': '31901100',
  'modalidade_empenho': 'E',
  'descricao_empenho': 'Valor que se empenha p/ fazer face as despesas com DISPENDIO COM O PESSOAL LOTADOS JUNTO A ESTA SECRETARIA DE CULTURA NO PRESENTE EXERCICIO FINANCEIRO DE 2017, NESTE MUNICIPIO DE ABAIARA.',
  'valor_anterior_saldo_dotacao': '59550.8000',
  'valor_empenhado': '59550.0000',
  'valor_atual_saldo_dotacao': '0.8000',
  'tipo_processo_licitatorio': 'F',
  'numero_documento_negociante': '66666666666',
  'estado_empenho': 'AP',
  'numero_nota_anulacao': ' ',
  'data_emissao_emp

### Criar CSV

In [56]:
cabecalhos = empenhos[0].keys()
cabecalhos

dict_keys(['codigo_municipio', 'exercicio_orcamento', 'codigo_orgao', 'codigo_unidade', 'data_emissao_empenho', 'numero_empenho', 'data_referencia_empenho', 'codigo_funcao', 'codigo_subfuncao', 'codigo_programa', 'codigo_projeto_atividade', 'numero_projeto_atividade', 'numero_subprojeto_atividade', 'codigo_elemento_despesa', 'modalidade_empenho', 'descricao_empenho', 'valor_anterior_saldo_dotacao', 'valor_empenhado', 'valor_atual_saldo_dotacao', 'tipo_processo_licitatorio', 'numero_documento_negociante', 'estado_empenho', 'numero_nota_anulacao', 'data_emissao_empenho_substituto', 'numero_empenho_substituto', 'cd_cpf_gestor', 'cpf_gestor_contrato', 'data_contrato', 'numero_contrato', 'data_realizacao_licitacao', 'numero_licitacao', 'codigo_tipo_negociante', 'nome_negociante', 'endereco_negociante', 'fone_negociante', 'cep_negociante', 'nome_municipio_negociante', 'codigo_uf', 'tipo_fonte', 'codigo_fonte', 'situacao_favorecido', 'data_situacao', 'atividade_principal_texto', 'atividade_pr

In [57]:
# Cria o arquivo.
#1 - 8161
arquivo_csv = open('empenhos_ce3.csv', 'w', newline='', encoding='utf-8')

# Cria o objeto de gravação
escrever = csv.writer(arquivo_csv, delimiter=',')

# Grava os cabecalhos
escrever.writerow(cabecalhos)

for linha in empenhos:
    escrever.writerow(linha.values())

# Fecha o arquivo.
arquivo_csv.close()

In [63]:
df = pd.read_csv("empenhos_ce3.csv")
df.iloc[19000:19050, :]

,codigo_municipio,exercicio_orcamento,codigo_orgao,codigo_unidade,data_emissao_empenho,numero_empenho,data_referencia_empenho,codigo_funcao,codigo_subfuncao,codigo_programa,...,fone_negociante,cep_negociante,nome_municipio_negociante,codigo_uf,tipo_fonte,codigo_fonte,situacao_favorecido,data_situacao,atividade_principal_texto,atividade_principal_codigo
19000,165,201900,11,1,2019-01-02 00:00:00,02010137,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19001,165,201900,11,1,2019-01-02 00:00:00,02010138,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19002,165,201900,11,1,2019-01-02 00:00:00,02010139,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19003,165,201900,11,1,2019-01-02 00:00:00,02010150,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19004,165,201900,11,1,2019-01-02 00:00:00,02010175,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19005,165,201900,11,1,2019-01-02 00:00:00,02010192,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19006,165,201900,11,1,2019-01-02 00:00:00,02010203,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19007,165,201900,11,1,2019-01-02 00:00:00,02010204,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19008,165,201900,11,1,2019-01-02 00:00:00,02010205,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN
19009,165,201900,11,1,2019-01-02 00:00:00,02010214,201901,13,392,42,...,,63750000,Tamboril,CE,1,1000000,NaN,NaN,NaN,NaN


In [59]:
df.shape

(20745, 44)